In [49]:
import requests
import time
import pandas as pd
import re
from datetime import time
from datetime import datetime
import pytz
import xml.etree.ElementTree as ET
from xml.dom import minidom


In [63]:
df_courses = pd.read_csv("courses_2024_fall_cleaned.csv")

In [64]:
def getTime(time_str):
    hr = int(time_str[0:2])
    min = int(time_str[3:5])
    tz = pytz.timezone("America/Chicago")
    time_obj = time(hr, min, 0, 0, tzinfo=tz)
    return time_obj

# convert a time object into a string object
def time_to_string(t):
    return t.strftime("%H:%M")

In [65]:
# convert a time string into a time object
def getTime(time_str):
    hr = int(time_str[0:2])
    min = int(time_str[3:5])
    tz = pytz.timezone("America/Chicago")
    time_obj = time(hr, min, 0, 0, tzinfo=tz)
    return time_obj

# convert a time object into a string object
def time_to_string(t):
    return t.strftime("%H:%M")

# query by building
def getCoursesInBuilding(building, df):
    df_building = df[df["Building"] == building]
    return df_building

# query all classes that occur today
def getClassToday(df, curr_datetime):
    weekdays = ['M', 'T', 'W', 'R', 'F', 'S', 'S']
    day_of_week = weekdays[curr_datetime.weekday()]
    df_today = df[df["Days"].str.contains(day_of_week)]
    return df_today

# get all courses that are going on right now
def getClassNow(df, building):
    df = getCoursesInBuilding(building, df)

    curr_datetime = datetime.now()
    df_today = getClassToday(df, curr_datetime)
    # get all courses that going on right now
    curr_time = curr_datetime.time()
    df_now = df_today[(df_today["StartTime"].apply(lambda x: getTime(x)) <= curr_time) & (df_today["EndTime"].apply(lambda x: getTime(x)) >= curr_time)]
    return df_now

# return all courses that occur in the next hour
def getClassSoon(df, building):
    df = getCoursesInBuilding(building, df)

    curr_datetime = datetime.now()
    df_today = getClassToday(df, curr_datetime)
    # get all courses that start within the next hour
    curr_time = curr_datetime.time()
    next_hr = time((curr_time.hour + 1) % 24, curr_time.minute, curr_time.second, curr_time.microsecond, curr_time.tzinfo)
    df_soon = df_today[(df_today["StartTime"].apply(lambda x: getTime(x)) <= next_hr) & (df_today["EndTime"].apply(lambda x: getTime(x)) >= next_hr)]
    return df_soon

# get all rooms that are currently open as well as when they will be taken again
def getOpenNow(df, building):
    df = getCoursesInBuilding(building, df)

    # get all courses going on today
    curr_datetime = datetime.now()
    df_today = getClassToday(df, curr_datetime)
    # get all courses going on right now
    df_now = getClassNow(df, building)
    # create set of all rooms in the target building
    rooms = set(df["RoomNumber"].unique())
    # filter out all rooms that are not open
    rooms_used = set(df_now["RoomNumber"].unique())
    rooms_open = rooms - rooms_used
    # loop through classes in the building today and return the next course time for each open room
    rooms_next_used = {room: time(23, 59) for room in rooms_open}   # 11:59 means room is open for rest of day
    for idx, row in df_today.iterrows():
        startTime = getTime(row["StartTime"])
        rm_num = row["RoomNumber"]
        # if the room is currently open and this class will occur later in the day
        if((rm_num in rooms_open) & (startTime > curr_datetime.time())):
            # find the soonest course
            if(startTime < rooms_next_used[rm_num]):
                rooms_next_used[rm_num] = startTime
    rooms_next_used_str = {room: time_to_string(t) for room, t in rooms_next_used.items()}
    return rooms_next_used_str

In [66]:
# df_testing = getClassNow(df_courses, "Education Building")
df_testing = getOpenNow(df_courses, "Education Building")
df_testing

{'37': '23:59',
 '166': '23:59',
 '162': '23:59',
 '210A': '23:59',
 '4F': '23:59',
 '188S': '23:59',
 '2': '23:59',
 '376': '23:59',
 '4G': '23:59',
 '389': '23:59',
 '42A': '23:59',
 '385': '23:59',
 '323': '23:59',
 '22': '23:59',
 '17': '23:59',
 '242': '23:59',
 '382': '23:59'}

In [69]:
df_courses.loc[df_courses["StartTime"].apply(lambda x: getTime(x)).idxmax()]

Subject                        AAS
CourseID                       355
Name           Race and Mixed Race
CRN                          61320
SectionCode                      A
Type            Lecture-Discussion
Days                            TR
RoomNumber                     219
Building         David Kinley Hall
Instructors            ['Lule, L']
StartTime                 12:30:00
EndTime                   01:50:00
Name: 33, dtype: object